In [10]:
import pandas as pd
import re

# Load datasets
building_path = '/Users/yangyungchyi/Documents/Spring 2025/MIS 462/Building_Type_of_Construction.csv'
business_path = '/Users/yangyungchyi/Documents/Spring 2025/MIS 462/0420_without_building.csv'

building_df = pd.read_csv(building_path)
business_df = pd.read_csv(business_path)

# === Street abbreviation mapping ===
abbreviation_map = {
    r"\bRDG\b": "RIDGE",
    r"\bRI\b": "RIDGE",
    r"\bRD\b": "ROAD",
    r"\bST\b": "STREET",
    r"\bAVE\b": "AVENUE",
    r"\bBLVD\b": "BOULEVARD",
    r"\bCTR\b": "CENTER",
    r"\bLN\b": "LANE",
    r"\bDR\b": "DRIVE",
    r"\bPL\b": "PLACE",
    r"\bPKWY\b": "PARKWAY",
    r"\bHWY\b": "HIGHWAY",
}

def normalize_address(addr):
    addr = str(addr).upper()
    addr = re.sub(r'[^\w\s]', '', addr)  # Remove punctuation
    for pattern, replacement in abbreviation_map.items():
        addr = re.sub(pattern, replacement, addr)
    return addr.strip()

# Apply normalization
building_df["Address_clean"] = building_df["Address:"].fillna("").apply(normalize_address)
business_df["Street1_clean"] = business_df["Street1"].fillna("").apply(normalize_address)

# === Merge on cleaned street fields ===
merged = pd.merge(
    business_df,
    building_df[["Address_clean", "Cleaned Group(s)"]],
    left_on="Street1_clean",
    right_on="Address_clean",
    how="left"
)

# Drop helper columns
merged = merged.drop(columns=["Street1_clean", "Address_clean"])

# Filter only B and F if needed
merged_filtered = merged[merged["Cleaned Group(s)"].isin(["B", "F"])]

# Remove duplicates
merged_filtered = merged_filtered.drop_duplicates()

# Save the result
output_path = '/Users/yangyungchyi/Documents/Spring 2025/MIS 462/0420_building.csv'
merged_filtered.to_csv(output_path, index=False)

print(f"✅ Cleaned & matched dataset saved to: {output_path}")


/var/folders/vl/66bj5_5x56n6mnk25p2th_m80000gn/T/ipykernel_40263/1733004996.py:8: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  building_df = pd.read_csv(building_path)
/var/folders/vl/66bj5_5x56n6mnk25p2th_m80000gn/T/ipykernel_40263/1733004996.py:9: DtypeWarning: Columns (17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  business_df = pd.read_csv(business_path)


✅ Cleaned & matched dataset saved to: /Users/yangyungchyi/Documents/Spring 2025/MIS 462/0420_building.csv
